In [353]:
import os
os.getcwd()

'/home/jovyan/work/running'

In [354]:
'''

Application 主架構

'''

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("./line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/home/jovyan/work/running" , static_folder = "/home/jovyan/work/running")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [355]:
'''

設定訂閱頻道後顯示的圖文
製作文字與圖片的教學訊息

'''
# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 消息清單
reply_message_list = [
    TextSendMessage(text="您好! 歡迎使用我們的頻道"),
    TextSendMessage(text="RunningAI可以協助您找出適合的運動路線、並讓您在運動時有更好的體驗!"),
    ImageSendMessage(original_content_url='%s/home/jovyan/work/running/girl.jpg' %server_url,
    preview_image_url='%s/home/jovyan/work/running/girl.jpg' %server_url)
]

In [356]:
'''

撰寫用戶關注時，我們要處理的商業邏輯

1. 取得用戶個資，並存回伺服器
2. 把先前製作好的自定義菜單，與用戶做綁定
3. 回應用戶，歡迎用的文字消息與圖片消息

'''


# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

# 載入requests套件
import requests


# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內
    with open("users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\r\n')
        
        
    # 將菜單綁定在用戶身上
    linkRichMenuId=secretFileContentJson.get("rich_menu_id")
    linkMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu/%s' % (event.source.user_id, linkRichMenuId)
    linkMenuRequestHeader={'Content-Type':'image/png','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}
    lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)
    
    # 回覆文字消息與圖片消息
    line_bot_api.reply_message(
        event.reply_token,
        reply_message_list
    )

In [357]:
'''
Button篇
    設定模板消息，指定其參數細節。

'''


#引入所需要的消息與模板消息
from linebot.models import (
    MessageEvent, TemplateSendMessage , PostbackEvent
)

#引入按鍵模板
from linebot.models.template import(
    ButtonsTemplate
)


'''
alt_text: Line簡覽視窗所出現的說明文字
template: 所使用的模板
ButtonsTemplate: 按鍵模板
    thumbnail_image_url: 展示圖片
    title: 標題
    text: 說明文字
    actions: 模板行為所使用的行為
    data: 觸發postback後用戶回傳值，可以對其做商業邏輯處理

'''
buttons_template_message = TemplateSendMessage(
    alt_text='Buttons template',
    template=ButtonsTemplate(
    title= "請選擇運動類型",
    text= "請選擇運動類型",
    actions= [
      {
        "type": "postback",
        "label": "騎車",
        "text": "騎車",
        "data": "騎車"
      },
      {
        "type": "postback",
        "label": "健走",
        "text": "健走",
        "data": "健走"
      },
      {
        "type": "postback",
        "label": "跑步",
        "text": "跑步",
        "data": "跑步"
      }
    ],

)
)

In [358]:
#引入所需要的消息與模板消息
from linebot.models import (
    MessageEvent, TemplateSendMessage , PostbackEvent
)
#引入按鍵模板
from linebot.models.template import(
    ButtonsTemplate
)

buttons_template_message1 = TemplateSendMessage(
    alt_text='Buttons template',
    template=ButtonsTemplate(
    title= "請選擇地區",
    text= "請選擇地區",
    actions= [
      {
        "type": "postback",
        "label": "北部",
        "text": "北部",
        "data": "北部"
      },
      {
        "type": "postback",
        "label": "中部",
        "text": "中部",
        "data": "中部"
      },
      {
        "type": "postback",
        "label": "南部",
        "text": "南部",
        "data": "南部"
      },
      {
        "type": "postback",
        "label": "東部或離島",
        "text": "東部或離島",
        "data": "東部或離島"
      }
    ],

)
)

In [359]:
jsonObject = {
  "type": "template",
  "altText": "this is a carousel template",
  "template": {
    "type": "carousel",
    "actions": [],
    "columns": [
      {
        "thumbnailImageUrl": "https://i.imgur.com/2hYqSLd.png",
        "title": "賽事1",
        "text": "復仇者聯盟路跑活動",
        "actions": [
          {
            "type": "uri",
            "label": "活動網址 1",
            "uri": "http://www.bao-ming.com/eb/www/activity_content.php?activitysn=4070"
          },
          {
            "type": "message",
            "label": "Action 2",
            "text": "Action 2"
          },
          {
            "type": "message",
            "label": "Action 3",
            "text": "Action 3"
          }
        ]
      },
      {
        "thumbnailImageUrl": "https://cdn-static.tibame.com/resize/entity-course/11/eeba1bb7-dc69-4a62-919f-cb3890368ca0_linebot_2x.png",
        "title": "Title",
        "text": "Text",
        "actions": [
          {
            "type": "message",
            "label": "Action 1",
            "text": "Action 1"
          },
          {
            "type": "message",
            "label": "Action 2",
            "text": "Action 2"
          },
          {
            "type": "message",
            "label": "Action 3",
            "text": "Action 3"
          }
        ]
      }
    ]
  }
}

In [360]:
#引入所需要的消息與模板消息
from linebot.models import (
    MessageEvent, TemplateSendMessage , PostbackEvent
)
#引入按鍵模板
from linebot.models.template import(
    CarouselTemplate
)

carousel_template_message = TemplateSendMessage.new_from_json_dict(jsonObject)
print(carousel_template_message)

{"altText": "this is a carousel template", "template": {"columns": [{"actions": [{"label": "\u6d3b\u52d5\u7db2\u5740 1", "type": "uri", "uri": "http://www.bao-ming.com/eb/www/activity_content.php?activitysn=4070"}, {"label": "Action 2", "text": "Action 2", "type": "message"}, {"label": "Action 3", "text": "Action 3", "type": "message"}], "text": "\u5fa9\u4ec7\u8005\u806f\u76df\u8def\u8dd1\u6d3b\u52d5", "thumbnailImageUrl": "https://i.imgur.com/2hYqSLd.png", "title": "\u8cfd\u4e8b1"}, {"actions": [{"label": "Action 1", "text": "Action 1", "type": "message"}, {"label": "Action 2", "text": "Action 2", "type": "message"}, {"label": "Action 3", "text": "Action 3", "type": "message"}], "text": "Text", "thumbnailImageUrl": "https://cdn-static.tibame.com/resize/entity-course/11/eeba1bb7-dc69-4a62-919f-cb3890368ca0_linebot_2x.png", "title": "Title"}], "type": "carousel"}, "type": "template"}


In [361]:
'''
設計一個字典

'''
template_message_dict = {
    "活動賽事":buttons_template_message
}
template_message_dict1 = {
    '騎車':buttons_template_message1,
    '健走':buttons_template_message1,
    '跑步':buttons_template_message1,
    '北部':carousel_template_message,
    '中部':carousel_template_message,
    '南部':carousel_template_message,
    '東部或離島':carousel_template_message
}

In [362]:
# 用戶發出文字消息時， 按條件內容, 回傳照片地圖
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text)
    )

In [363]:
#用戶點擊button後，觸發postback event，對其回傳做相對應處理

@handler.add(PostbackEvent)
def handle_post_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    print(event.postback.data)
    line_bot_api.reply_message(
    event.reply_token,
    template_message_dict1.get(event.postback.data)
    )

In [364]:
'''

執行此句，啟動Server，觀察後，按左上方塊，停用Server

'''

if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.20.0.2 - - [24/Oct/2019 13:27:55] "POST / HTTP/1.1" 200 -


騎車


172.20.0.2 - - [24/Oct/2019 13:27:59] "POST / HTTP/1.1" 200 -
[2019-10-24 13:27:59,894] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/conda/lib/python3.7/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1598, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-354-861791efa2cd>", line 46,

南部


172.20.0.2 - - [24/Oct/2019 13:28:02] "POST / HTTP/1.1" 200 -
[2019-10-24 13:28:03,329] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/conda/lib/python3.7/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1598, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-354-861791efa2cd>", line 46,

...

[2019-10-24 14:27:21,182] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/conda/lib/python3.7/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1598, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-354-861791efa2cd>", line 46, in callback
    handler.handle(body, signature)
  File "/opt/

北部


[2019-10-24 14:27:22,683] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/conda/lib/python3.7/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1598, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-354-861791efa2cd>", line 46, in callback
    handler.handle(body, signature)
  File "/opt/

南部


172.20.0.2 - - [24/Oct/2019 14:27:25] "POST / HTTP/1.1" 200 -
[2019-10-24 14:27:26,893] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/conda/lib/python3.7/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1598, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-354-861791efa2cd>", line 46,